# Построение моели рекомендации тарифа

В вашем распоряжении данные о поведении клиентов, которые уже перешли на тарифы «Смарт» и «Ультра». Нужно построить модель для задачи классификации, которая проанализирует поведение клиентов и предложит пользователям новый тариф: «Смарт» или «Ультра». Нам так же необходимо довести долю правильных ответов по крайней мере до 0.75 на тестовой выборке.

*Описание данных:*

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:   

 - `сalls` — количество звонков   
 - `minutes` — суммарная длительность звонков в минутах   
 - `messages` — количество sms-сообщений   
 - `mb_used` — израсходованный интернет-трафик в Мб   
 - `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение данных</a></span></li><li><span><a href="#Разделение-на-выборки" data-toc-modified-id="Разделение-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделение на выборки</a></span></li><li><span><a href="#Исследование-качества-разных-моделей" data-toc-modified-id="Исследование-качества-разных-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование качества разных моделей</a></span><ul class="toc-item"><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Проверка-качества-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-качества-модели-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка качества модели на тестовой выборке</a></span></li><li><span><a href="#Проверка-модели-на-вменяемость" data-toc-modified-id="Проверка-модели-на-вменяемость-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка модели на вменяемость</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

## Изучение данных   
Загрузим необходимые для нас библиотеки, а так же загрузим и изучим данные из файла `users_behavior.csv`

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
pd.options.display.float_format = '{:,.2f}'.format
print('Setup complete')

Setup complete


In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('datasets/users_behavior.csv')
display(df.head(), df.info(), df.describe())
print('Кол-во пропусков:\n', df.isna().sum())
print()
print('Кол-во дубликатов:', df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.00,311.90,83.00,"19,915.42",0
1,85.00,516.75,56.00,"22,696.96",0
2,77.00,467.66,86.00,"21,060.45",0
3,106.00,745.53,81.00,"8,437.39",1
4,66.00,418.74,1.00,"14,502.75",0


None

,calls,minutes,messages,mb_used,is_ultra
count,"3,214.00","3,214.00","3,214.00","3,214.00","3,214.00"
mean,63.04,438.21,38.28,"17,207.67",0.31
std,33.24,234.57,36.15,"7,570.97",0.46
min,0.00,0.00,0.00,0.00,0.00
25%,40.00,274.58,9.00,"12,491.90",0.00
50%,62.00,430.60,30.00,"16,943.24",0.00
75%,82.00,571.93,57.00,"21,424.70",1.00
max,244.00,"1,632.06",224.00,"49,745.73",1.00


Кол-во пропусков:
 calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Кол-во дубликатов: 0


В данных всё в порядке. Всего имеется 3214 записей, пропуски отсутсвуют, дубликаты отсутсвуют.

## Разделение на выборки   
Нам необходимо разделить нашу выборку на три части: Обучающую, валидационную и тестовую.   
Для начала посмотрим на распределение тарифов абонентов в нашей выборке.

In [3]:
print(f'Доля абонентов Ультра: {(len(df.query("is_ultra == 1")) / len(df)):.1%}')
print(f'Доля абонентов Смарт: {(len(df.query("is_ultra == 0")) / len(df)):.1%}')

Доля абонентов Ультра: 30.6%
Доля абонентов Смарт: 69.4%


Извлечем из наших данных целевой признак и признаки, присвоим им названия `target` и `features` соответсвенно.   
Выделим тестовую выборку из датасета. Для удобства сохраним значение `random_state` в переменной `state`.

In [4]:
state = 43
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']
features_df, features_test, target_df, target_test = train_test_split(
    features, target, test_size=0.2, random_state=state)

Разделим оставшуюся выборку на тренировочную и валидационную.

In [5]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_df, target_df, test_size=0.25, random_state=state)

Проверим правильность разбиения получившихся выборок.

In [6]:
print(f'Размер обучающей выборки составляет: {features_train.shape[0]  / len(df):.1%}')
print(f'Размер тестовой выборки составляет: {features_test.shape[0] / len(df):.1%}')
print(f'Размер валидационной выборки составляет: {features_valid.shape[0] / len(df):.1%}')

Размер обучающей выборки составляет: 60.0%
Размер тестовой выборки составляет: 20.0%
Размер валидационной выборки составляет: 20.0%


Проверим так же процент пользователей тарифа Ультра в каждой из полученных выборок.

In [7]:
print(f'Доля абонентов Ультра в обучающей выборке: {(target_train.sum() / len(target_train)):.1%}')
print(f'Доля абонентов Ультра в тестовой выборке: {(target_test.sum() / len(target_test)):.1%}')
print(f'Доля абонентов Ультра в валидационной выборки: {(target_valid.sum() / len(target_valid)):.1%}')

Доля абонентов Ультра в обучающей выборке: 30.8%
Доля абонентов Ультра в тестовой выборке: 31.1%
Доля абонентов Ультра в валидационной выборки: 29.9%


Выборки разделены верно 60%, 20%, 20%. Доля абонентов тарифа Ультра примерно во всех выборках одинаковая.

## Исследование качества разных моделей   
Здесь мы исследуем три известных нам модели и выберем наилучшую.   
*Исследуемые модели:*   
 - Дерево решений
 - Случайный лес
 - Логистическая регрессия

### Дерево решений

In [8]:
best_dtc_model = None
best_result = 0
best_depth = 0
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=state, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_dtc_model = model
        best_result = result
        best_depth = depth
        
print("Accuracy лучшей модели на валидационной выборке:", best_result, "Глубина дерева:", best_depth)

Accuracy лучшей модели на валидационной выборке: 0.7993779160186625 Глубина дерева: 11


### Случайный лес

In [9]:
best_rfc_model = None
best_result = 0
best_est = 0
for est in range(1, 50):
    model = RandomForestClassifier(random_state=state, n_estimators=est) 
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_rfc_model = model
        best_result = result
        best_est = est
        
print("Accuracy наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est)

Accuracy наилучшей модели на валидационной выборке: 0.8227060653188181 Количество деревьев: 47


### Логистическая регрессия

In [10]:
model_lr = LogisticRegression(random_state=state, solver='lbfgs', max_iter=1000)
model_lr.fit(features_train, target_train)

predictions_valid = model_lr.predict(features_valid)
valid_acc = accuracy_score(target_valid, predictions_valid)

print("Accuracy модели логистической регрессии на валидационной выборке:", valid_acc)

Accuracy модели логистической регрессии на валидационной выборке: 0.7465007776049767


Запишем наши полученные значения:   
 1. Лучшая модель дерева решений 0.7993779160186625 Глубина дерева: 11   
 2. Лучшая модель случайного леса 0.8227060653188181 Количество деревьев: 47   
 3. Логистическая регрессия 0.7465007776049767   
 
Теперь на предстоит проверить наши модели на тестовой выборке.

## Проверка качества модели на тестовой выборке
Самый лучший результат мы получили на модели случайного леса. Проверим каждую из этих моделей на тестовой выборке.

In [11]:
models = {best_dtc_model: 'Дерева решений',
         best_rfc_model: 'Случайного леса',
         model_lr: 'Логистической регрессии'}

for model in models:
    predictions_test = model.predict(features_test)
    result = accuracy_score(target_test, predictions_test)
    print(f'Accuracy выбранной модели {models.get(model)} на тестовой выборке: {result}')

Accuracy выбранной модели Дерева решений на тестовой выборке: 0.7776049766718507
Accuracy выбранной модели Случайного леса на тестовой выборке: 0.8133748055987559
Accuracy выбранной модели Логистической регрессии на тестовой выборке: 0.7667185069984448


Лучшую точность нам показала модель случайного леса 0.81, что хоть и не намного, но всё же больше 0.75.

## Проверка модели на вменяемость   
Проверим нашу модель на вменяемость проверив долю пользователей тарифа Ультра в предсказанных значениях.

In [12]:
predictions_test = best_rfc_model.predict(features_test)
print(f'Доля абонентов Ультра в тестовой выборке: {(target_test.sum() / len(target_test)):.1%}')
print(f'Доля абонентов Ультра в передсказанной выборке: {(predictions_test.sum() / len(predictions_test)):.1%}')

Доля абонентов Ультра в тестовой выборке: 31.1%
Доля абонентов Ультра в передсказанной выборке: 24.6%


Модель предсказала практически тоже самое кол-во клиентов с тарифом Ультра.

## Общий вывод   
Самой лучшей моделью для данной выборки стал случайный лес, который показал точность тестовой модели в 0.81 и прошёл проверку на вменяемость.